In [106]:
import pandas as pd
import numpy as np
import dateutil
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
import re
from seaborn import plt
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 90)
pd.set_option('display.max_columns', 90)
pd.set_option('display.precision', 10)

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt

In [120]:
df = pd.read_csv('imdb.csv')
df = df.drop('Unnamed: 0', axis = 1)

In [121]:
df.head()

,title,gross,budget,rating,numvote,genre,mpaa,date,runtime,aspect
0,The Birth of a Nation,3000000,100000,6.8,17162,"['Drama', 'History', 'War']",Not Rated,1915-03-03,190,1.33
1,The Kid,2500000,250000,8.3,68767,"['Comedy', 'Drama', 'Family']",Not Rated,1921-02-06,68,1.33
2,Ben-Hur: A Tale of the Christ,9000000,3900000,8.0,6107,"['Adventure', 'Drama', 'Romance']",Not Rated,1927-10-08,143,1.33
3,The Jazz Singer,3000000,422000,6.8,6866,"['Drama', 'Music', 'Musical', 'Romance']",Unrated,1927-10-06,88,1.33
4,Frankenstein,12000000,291000,8.0,48844,"['Drama', 'Horror', 'Sci-Fi']",Not Rated,1931-11-21,70,1.37


In [125]:
# set dummy variables
count = 0
for genre_item in df['genre']:
    str1 = genre_item[1:-1]
    l2 = str1.split(',')
    genre_sublist = [re.search('.*\'(.*)\'.*', str2).group(1) for str2 in l2]
    for gen1 in genre_sublist:
        df.loc[count, gen1] = 1 
    count += 1

# remove NaNs
df = df.replace(np.NAN, 0)

In [122]:
# old method -- set dummy variables
# create genre_sublist to be added into
genre_list = []

count = 0
for genre_item in df['genre']:
    str1 = genre_item[1:-1]
    l2 = str1.split(',')
    genre_sublist = [re.search('.*\'(.*)\'.*', str2).group(1) for str2 in l2]
    
    #df.loc[count, 'genre'] = genre_sublist
    count += 1
    genre_list.extend(genre_sublist)

genre_set = set(genre_list)
genre_list = list(genre_set)

# create new columns
for col in genre_list:
    df[col] = 0

# set dummy variables
count = 0
for genre_item in df['genre']:
    str1 = genre_item[1:-1]
    l2 = str1.split(',')
    genre_sublist = [re.search('.*\'(.*)\'.*', str2).group(1) for str2 in l2]
    for gen1 in genre_sublist:
        df.loc[count, gen1] = 1 
    count += 1
    

In [128]:
df.to_csv('imdb212.csv')

In [127]:
df.corr()

,gross,budget,rating,numvote,runtime,aspect,Mystery,Romance,Sport,Sci-Fi,Family,Horror,Thriller,Film-Noir,Musical,Drama,Fantasy,Western,Animation,Music,Adventure,Crime,Action,Comedy,War,Biography,History
gross,1.0000000000,0.6378603282,0.2125559560,0.6299557706,0.2290245319,0.1011399417,-0.0319653646,-0.0575249743,-0.0020701434,0.1492182294,0.2063865810,-0.1073746602,-0.0462394480,-0.0222812555,0.0197373636,-0.2003897701,0.1755479312,-0.0146667614,0.2006386062,-0.0245960295,0.3282690930,-0.0897230742,0.1985896398,0.0108240271,-0.0122876919,-0.0552773738,-0.0290000011
budget,0.6378603282,1.0000000000,0.0174846981,0.4193695400,0.2681142202,0.2501490231,-0.0181091925,-0.0982366720,-0.0076026596,0.2283782762,0.2195337634,-0.1474480925,0.0248136975,-0.0268444754,-0.0224855916,-0.2088840387,0.2047950943,0.0158566468,0.2307836670,-0.0635327606,0.4313116213,-0.0627427781,0.3416299041,-0.0664084326,0.0245841997,-0.0586643529,0.0283198132
rating,0.2125559560,0.0174846981,1.0000000000,0.4736277927,0.4015123762,0.0021496521,0.0120622726,-0.0016104154,0.0126594182,-0.0606359949,-0.0694397569,-0.2078208406,-0.0593883483,0.0442378758,0.0272746808,0.3215502809,-0.0670663480,0.0446233533,0.0744405598,-0.0068214115,-0.0145310438,0.0470845632,-0.1011979495,-0.1767941961,0.1070812012,0.1648430149,0.1177603622
numvote,0.6299557706,0.4193695400,0.4736277927,1.0000000000,0.3298628533,0.1360862436,0.0217770116,-0.1052789773,-0.0397073532,0.1617133550,-0.0139003901,-0.0823878670,0.0231058734,-0.0093096768,-0.0289008939,-0.0496014157,0.0687049192,-0.0088799058,0.0453753252,-0.0641231514,0.1816812195,-0.0046357108,0.1503437697,-0.1354578884,0.0143949796,-0.0059099570,-0.0054061358
runtime,0.2290245319,0.2681142202,0.4015123762,0.3298628533,1.0000000000,0.2125449259,0.0072986668,0.0397468126,0.0269667674,-0.0192554344,-0.1913110858,-0.2008326635,0.0059559010,-0.0168545160,-0.0084612347,0.3198087528,-0.0916684601,0.0942929072,-0.2078467326,0.0058663948,0.0611767433,0.0147796509,0.0661099055,-0.3123614828,0.2086491664,0.2041623819,0.2942473012
aspect,0.1011399417,0.2501490231,0.0021496521,0.1360862436,0.2125449259,1.0000000000,0.0681716834,-0.0892726480,-0.0131733684,0.0714696137,-0.1077914232,-0.0108706966,0.1623295117,-0.0527737895,-0.0511558212,0.0059033669,-0.0309553539,0.0555797033,-0.1053483773,-0.0374137755,0.1106037732,0.0623458563,0.2317391066,-0.2142776380,0.0649844216,0.0436841196,0.0668139268
Mystery,-0.0319653646,-0.0181091925,0.0120622726,0.0217770116,0.0072986668,0.0681716834,1.0000000000,-0.1005362215,-0.0687091639,0.0375821895,-0.0675469415,0.1480862492,0.3159324990,0.0373650210,-0.0505622989,-0.0105646536,-0.0271804822,-0.0306508816,-0.0372335476,-0.0563239343,-0.0753709567,0.1398708125,-0.0527956097,-0.2089850372,-0.0483304782,-0.0812839085,-0.0526899322
Romance,-0.0575249743,-0.0982366720,-0.0016104154,-0.1052789773,0.0397468126,-0.0892726480,-0.1005362215,1.0000000000,-0.0314299141,-0.1282003352,-0.0475417945,-0.1711020666,-0.2352336802,0.0191805938,0.0956660192,0.1641201875,-0.0268847589,-0.0016555423,-0.0492874847,0.0848658687,-0.1204434438,-0.1550658074,-0.1987532091,0.1730662489,-0.0066613635,-0.0050395302,-0.0074502731
Sport,-0.0020701434,-0.0076026596,0.0126594182,-0.0397073532,0.0269667674,-0.0131733684,-0.0687091639,-0.0314299141,1.0000000000,-0.0508799450,0.0656717324,-0.0699866818,-0.1188148862,-0.0058895947,-0.0320003848,0.0527365737,-0.0500779988,-0.0158258058,-0.0064298535,-0.0384429008,-0.0560528925,-0.0821377271,-0.0424973582,0.0098728032,-0.0317023010,0.1063480769,0.0043692457
Sci-Fi,0.1492182294,0.2283782762,-0.0606359949,0.1617133550,-0.0192554344,0.0714696137,0.0375821895,-0.1282003352,-0.0508799450,1.0000000000,0.0344103229,0.0959102860,0.1171842362,-0.0117812421,-0.0466770167,-0.2227392879,-0.0043315620,-0.0349681227,0.0414068900,-0.0586456876,0.2241884605,-0.1435919923,0.2794387965,-0.1264156859,-0.0693321434,-0.0912661891,-0.0736713724
